## Trying out basic FIRE calculations

# Taking SIP input

In [46]:
import ipywidgets as widgets
# from IPython.display import display, VBox, HBox
from IPython.display import display
from ipywidgets import VBox, HBox 


# Create slider and manual input
sip_slider = widgets.IntSlider(
    value=20000,
    min=1000,
    max=200000,
    step=1000,
    description='Monthly SIP (₹):',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='60%')
)

sip_input = widgets.BoundedIntText(
    value=20000,
    min=1000,
    max=200000,
    step=1000,
    description='Enter SIP (₹):',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='40%')
)

# Display output label
sip_display = widgets.Label()

# Sync slider and manual input
widgets.jslink((sip_slider, 'value'), (sip_input, 'value'))

# Update display label on change
def update_display(change):
    sip_display.value = f"👉 Your Monthly SIP is set to ₹{change['new']:,}"

sip_slider.observe(update_display, names='value')

# Display the UI
ui = VBox([
    HBox([sip_slider, sip_input]),
    sip_display
])
display(ui)

# Trigger initial display
sip_display.value = f"👉 Your Monthly SIP is set to ₹{sip_slider.value:,}"


# SIP table

In [49]:
import pandas as pd

# Get SIP value from the widget
monthly_sip = sip_slider.value
years = 30

# Prepare data for each year
data = []
for year in range(1, years + 1):
    data.append({
        "Year": year,
        "Monthly SIP (₹)": monthly_sip,
        "Yearly SIP (₹)": monthly_sip * 12
    })

# Convert to DataFrame
df_sip = pd.DataFrame(data)

# Format values
df_sip["Monthly SIP (₹)"] = df_sip["Monthly SIP (₹)"].map(lambda x: f"₹{x:,.0f}")
df_sip["Yearly SIP (₹)"] = df_sip["Yearly SIP (₹)"].map(lambda x: f"₹{x:,.0f}")

# Display
df_sip.style.set_caption("📅 Monthly & Yearly SIP Over 30 Years").set_table_styles([
    {'selector': 'caption', 'props': [('caption-side', 'top'), ('font-size', '16px'), ('font-weight', 'bold')]}
])


,Year,Monthly SIP (₹),Yearly SIP (₹)
0,1,"₹80,000","₹960,000"
1,2,"₹80,000","₹960,000"
2,3,"₹80,000","₹960,000"
3,4,"₹80,000","₹960,000"
4,5,"₹80,000","₹960,000"
5,6,"₹80,000","₹960,000"
6,7,"₹80,000","₹960,000"
7,8,"₹80,000","₹960,000"
8,9,"₹80,000","₹960,000"
9,10,"₹80,000","₹960,000"


### Display lumpsum cell


In [51]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import VBox, HBox

# Input: Lumpsum amount widget
lumpsum_input = widgets.IntText(
    value=300000,
    description='Total Lumpsum (₹):',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%')
)
display(lumpsum_input)

# ROI Input Widget
roi_input = widgets.FloatSlider(
    value=12.0,
    min=5.0,
    max=15.0,
    step=0.1,
    description='Expected ROI (%):',
    readout_format='.1f',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='70%')
)

display(roi_input)



IntText(value=300000, description='Total Lumpsum (₹):', layout=Layout(width='50%'), style=DescriptionStyle(des…

FloatSlider(value=12.0, description='Expected ROI (%):', layout=Layout(width='70%'), max=15.0, min=5.0, readou…

In [52]:

# Get values
monthly_sip = sip_slider.value
total_lumpsum = lumpsum_input.value
print( total_lumpsum,'ROI:',roi_input.value)
years = 30
lumpsum_per_year = total_lumpsum / years
monthly_from_lumpsum = lumpsum_per_year / 12

# Prepare table
data = []
for year in range(1, years + 1):
    combined_monthly = monthly_sip + monthly_from_lumpsum
    yearly_sip = combined_monthly * 12
    data.append({
        "Year": year,
        "Monthly SIP (₹)": round(monthly_sip),
        "Lumpsum/month (₹)": round(monthly_from_lumpsum),
        "Combined Monthly Invest (₹)": round(combined_monthly),
        "Yearly Total Invest (₹)": round(yearly_sip)
    })

df_sip_lumpsum = pd.DataFrame(data)

# Format
for col in df_sip_lumpsum.columns[1:]:
    df_sip_lumpsum[col] = df_sip_lumpsum[col].map(lambda x: f"₹{x:,.0f}")

# Display
df_sip_lumpsum.style.set_caption("📅 SIP + Lumpsum Spread Investment Table").set_table_styles([
    {'selector': 'caption', 'props': [('caption-side', 'top'), ('font-size', '16px'), ('font-weight', 'bold')]}
])


300000 ROI: 12.0


,Year,Monthly SIP (₹),Lumpsum/month (₹),Combined Monthly Invest (₹),Yearly Total Invest (₹)
0,1,"₹80,000",₹833,"₹80,833","₹970,000"
1,2,"₹80,000",₹833,"₹80,833","₹970,000"
2,3,"₹80,000",₹833,"₹80,833","₹970,000"
3,4,"₹80,000",₹833,"₹80,833","₹970,000"
4,5,"₹80,000",₹833,"₹80,833","₹970,000"
5,6,"₹80,000",₹833,"₹80,833","₹970,000"
6,7,"₹80,000",₹833,"₹80,833","₹970,000"
7,8,"₹80,000",₹833,"₹80,833","₹970,000"
8,9,"₹80,000",₹833,"₹80,833","₹970,000"
9,10,"₹80,000",₹833,"₹80,833","₹970,000"


# Adding bonus column

In [56]:
bonus_input = widgets.IntText(
    value=50000,
    description='Yearly Bonus (₹):',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%')
)

display(bonus_input)


IntText(value=50000, description='Yearly Bonus (₹):', layout=Layout(width='50%'), style=DescriptionStyle(descr…

In [55]:
# Parameters
monthly_sip = sip_slider.value
total_lumpsum = lumpsum_input.value
expected_annual_return = roi_input.value / 100
years = 30

# Precompute lumpsum and combined SIP
lumpsum_per_year = total_lumpsum / years
monthly_from_lumpsum = lumpsum_per_year / 12

# Track corpus
corpus = 0
data = []

for year in range(1, years + 1):
    combined_monthly = monthly_sip + monthly_from_lumpsum
    yearly_invest = combined_monthly * 12

    corpus = (corpus + yearly_invest) * (1 + expected_annual_return)

    data.append({
        "Year": year,
        "Monthly SIP (₹)": round(monthly_sip),
        "Lumpsum/month (₹)": round(monthly_from_lumpsum),
        "Yearly Investment (₹)": round(yearly_invest),
        "Corpus (₹)": round(corpus)
    })

df_corpus = pd.DataFrame(data)

# Format ₹ columns first
for col in df_corpus.columns[1:4]:  # Skip 'Corpus (₹)' for now
    df_corpus[col] = df_corpus[col].map(lambda x: f"₹{x:,.0f}")

# Format the Corpus (₹) column but keep it numeric for conversion
df_corpus["Corpus (₹)"] = df_corpus["Corpus (₹)"].map(lambda x: x)  # skip ₹ formatting for now

# ✅ Add Corpus in Crores column
df_corpus["Corpus (Cr ₹)"] = df_corpus["Corpus (₹)"] / 1e7
df_corpus["Corpus (Cr ₹)"] = df_corpus["Corpus (Cr ₹)"].map(lambda x: f"₹{x:.2f} Cr")

# Optional: drop Corpus (₹) if not needed
# df_corpus = df_corpus.drop(columns=["Corpus (₹)"])

# Display the styled table
df_corpus.style.set_caption("📊 Yearly Investment with Corpus in Cr ₹").set_table_styles([
    {'selector': 'caption', 'props': [('caption-side', 'top'), ('font-size', '16px'), ('font-weight', 'bold')]}
])

,Year,Monthly SIP (₹),Lumpsum/month (₹),Yearly Investment (₹),Corpus (₹),Corpus (Cr ₹)
0,1,"₹80,000",₹833,"₹970,000",1086400,₹0.11 Cr
1,2,"₹80,000",₹833,"₹970,000",2303168,₹0.23 Cr
2,3,"₹80,000",₹833,"₹970,000",3665948,₹0.37 Cr
3,4,"₹80,000",₹833,"₹970,000",5192262,₹0.52 Cr
4,5,"₹80,000",₹833,"₹970,000",6901733,₹0.69 Cr
5,6,"₹80,000",₹833,"₹970,000",8816341,₹0.88 Cr
6,7,"₹80,000",₹833,"₹970,000",10960702,₹1.10 Cr
7,8,"₹80,000",₹833,"₹970,000",13362387,₹1.34 Cr
8,9,"₹80,000",₹833,"₹970,000",16052273,₹1.61 Cr
9,10,"₹80,000",₹833,"₹970,000",19064946,₹1.91 Cr
